In [1]:
import pandas as pd
import math

In [81]:
df=pd.DataFrame(columns=['outlook','temp','humidity','windy','target'])
df.loc[0]=['rainy','hot','high','false','n']
df.loc[1]=['rainy','hot','high','true','n']
df.loc[2]=['overcast','hot','high','false','y']
df.loc[3]=['sunny','mild','high','false','y']
df.loc[4]=['sunny','cool','normal','false','y']
df.loc[5]=['sunny','cool','normal','true','n']
df.loc[6]=['overcast','cool','normal','true','y']
df.loc[7]=['rainy','mild','high','false','n']
df.loc[8]=['rainy','cool','normal','false','y']
df.loc[9]=['sunny','mild','normal','false','y']
df.loc[10]=['rainy','mild','normal','true','y']
df.loc[11]=['overcast','mild','high','true','y']
df.loc[12]=['overcast','hot','normal','false','y']
df.loc[13]=['sunny','mild','high','true','n']
n=14

In [3]:
def entropy(string):
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])
    return entropy

In [4]:
def infogain(l):
    ig = l[0]
    for d in l[1:]:
        ig-=d
    return ig

In [5]:
def nested_set(dic, keys, value):
    for key in keys[:-1]:
        dic = dic.setdefault(key, {})
    dic[keys[-1]] = value

In [83]:
dt={}
def fun(done,val):
    ss=""
    for i in range(0,n):
        fl=1
        for j in range(0,len(done)):
            if df[done[j]][i]==val[j]:
                continue
            else:
                fl=0
                break
        if fl==1:
            ss+=df['target'][i]
    ent=entropy(ss)
    lenn=len(ss)
    
    maxx=-500
    selc=""
    for c in list(df.columns):
        if (c not in done) & (c not in ['target']):
            ll=[ent]
            for k in dict.fromkeys(list(df[c])).keys():
                newdone=list(done)
                newval=list(val)
                newdone.append(c)
                newval.append(k)
                s=""
                for i in range(0,n):
                    fl=1
                    for j in range(0,len(newdone)):
                        if df[newdone[j]][i]==newval[j]:
                            continue
                        else:
                            fl=0
                            break
                    if fl==1:
                        s+=df['target'][i]
                ll.append(entropy(s)*len(s)/lenn)
            
            ig=infogain(ll)
            if(ig>maxx):
                maxx=ig
                selc=c
    if (maxx==-500)|(maxx==0):
        newdone=[]
        for i,j in zip(done,val):
            newdone.append(i)
            newdone.append(j)
        if len(newdone)==0:
            newdone.append(0)
        nested_set(dt,newdone,ss[0])
    else:
        newdone=[]
        newdone2=list(done)
        newdone2.append(selc)
        for i,j in zip(done,val):
            newdone.append(i)
            newdone.append(j)
        newdone.append(selc)
        nested_set(dt,newdone,{})
        for k in dict.fromkeys(list(df[selc])).keys():
            newval=list(val)
            newval.append(k)
            newdone1=list(newdone)
            newdone1.append(k)
            nested_set(dt,newdone1,{})
            fun(newdone2,newval)


In [84]:
def predict(dt1,l):
    key=list(dt1.keys())[0]
    while(1):
        dt1=dt1[key]
        if type(dt1) is not dict:
            break
        val=l[key]
        dt1=dt1[val]
        if type(dt1) is not dict:
            break
        key=list(dt1.keys())[0]
    return (dt1)

In [88]:
fun([],[])
print("The generated decision tree is: \n")
print(dt)
print("\nThe predicted value for the given data is: ")
print(predict(dt,{'outlook':'rainy','temp':'high','humidity':'normal','windy':'true'}))

The generated decision tree is: 

{'outlook': {'rainy': {'humidity': {'high': 'n', 'normal': 'y'}}, 'overcast': 'y', 'sunny': {'windy': {'false': 'y', 'true': 'n'}}}}

The predicted value for the given data is: 
y
